# M2 MacBook Compatibility Check

This notebook verifies that your M2 MacBook is properly configured for running the Efficient Gaussian Process on Graphs project with optimal performance.

## What we'll check:
1. PyTorch installation and MPS (Metal Performance Shaders) support
2. Basic tensor operations on GPU
3. Custom graph kernel compatibility
4. Memory usage and performance benchmarks
5. Required dependencies

In [3]:
import sys
import torch
import numpy as np
import platform
import psutil
import time
import subprocess
from pathlib import Path

print("=== SYSTEM INFORMATION ===")
print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")
print(f"Architecture: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"CPU cores: {psutil.cpu_count(logical=False)} physical, {psutil.cpu_count(logical=True)} logical")
print(f"Total RAM: {psutil.virtual_memory().total / (1024**3):.1f} GB")
print()

print("=== PYTORCH INFORMATION ===")
print(f"PyTorch version: {torch.__version__}")

# Check MPS support (Metal Performance Shaders for Apple Silicon)
try:
    mps_available = torch.backends.mps.is_available()
    mps_built = torch.backends.mps.is_built()
    print(f"MPS (Metal) available: {mps_available}")
    print(f"MPS built: {mps_built}")
except AttributeError:
    print("MPS support not available in this PyTorch version")
    mps_available = False
    mps_built = False

# Device selection for Mac
if mps_available:
    print("✅ MPS is available - your M2 MacBook is ready for GPU acceleration!")
    device = torch.device("mps")
else:
    print("⚠️  MPS not available - using CPU only")
    print("   Consider updating PyTorch: pip install torch torchvision torchaudio")
    device = torch.device("cpu")

print(f"Selected device: {device}")

=== SYSTEM INFORMATION ===
Python version: 3.11.5 (v3.11.5:cce6ba91b3, Aug 24 2023, 10:50:31) [Clang 13.0.0 (clang-1300.0.29.30)]
Platform: macOS-15.5-arm64-arm-64bit
Architecture: arm64
Processor: arm
CPU cores: 8 physical, 8 logical
Total RAM: 8.0 GB

=== PYTORCH INFORMATION ===
PyTorch version: 2.7.1
MPS (Metal) available: True
MPS built: True
✅ MPS is available - your M2 MacBook is ready for GPU acceleration!
Selected device: mps


In [4]:
print("=== TESTING BASIC TENSOR OPERATIONS ===")

# Test basic tensor creation and operations
sizes = [100, 1000, 5000]
results = {}

for size in sizes:
    print(f"\nTesting {size}x{size} matrices...")
    
    # CPU test
    start_time = time.time()
    a_cpu = torch.randn(size, size)
    b_cpu = torch.randn(size, size)
    c_cpu = torch.mm(a_cpu, b_cpu)
    cpu_time = time.time() - start_time
    
    # MPS test (if available)
    if device.type == "mps":
        start_time = time.time()
        a_gpu = torch.randn(size, size, device=device)
        b_gpu = torch.randn(size, size, device=device)
        
        # Synchronization for accurate timing
        torch.mps.synchronize()
            
        start_time = time.time()
        c_gpu = torch.mm(a_gpu, b_gpu)
        
        torch.mps.synchronize()
            
        gpu_time = time.time() - start_time
        speedup = cpu_time / gpu_time if gpu_time > 0 else 0
        
        print(f"  CPU time: {cpu_time:.4f}s")
        print(f"  MPS time: {gpu_time:.4f}s")
        print(f"  Speedup: {speedup:.2f}x")
        
        results[size] = {"cpu": cpu_time, "gpu": gpu_time, "speedup": speedup}
    else:
        print(f"  CPU time: {cpu_time:.4f}s")
        results[size] = {"cpu": cpu_time}

if device.type == "mps" and results:
    avg_speedup = np.mean([r["speedup"] for r in results.values()])
    print(f"\nAverage MPS speedup: {avg_speedup:.2f}x")
    if avg_speedup > 2.0:
        print("✅ Excellent MPS acceleration!")
    elif avg_speedup > 1.2:
        print("✅ Good MPS acceleration")
    else:
        print("⚠️  Limited MPS acceleration - check PyTorch MPS installation")
elif device.type == "cpu":
    print("\n⚠️  No GPU acceleration available - consider installing PyTorch with MPS support")

=== TESTING BASIC TENSOR OPERATIONS ===

Testing 100x100 matrices...
  CPU time: 0.0163s
  MPS time: 0.1242s
  Speedup: 0.13x

Testing 1000x1000 matrices...
  CPU time: 0.0229s
  MPS time: 0.0805s
  Speedup: 0.28x

Testing 5000x5000 matrices...
  CPU time: 0.0163s
  MPS time: 0.1242s
  Speedup: 0.13x

Testing 1000x1000 matrices...
  CPU time: 0.0229s
  MPS time: 0.0805s
  Speedup: 0.28x

Testing 5000x5000 matrices...
  CPU time: 0.7267s
  MPS time: 0.2606s
  Speedup: 2.79x

Average MPS speedup: 1.07x
⚠️  Limited MPS acceleration - check PyTorch MPS installation
  CPU time: 0.7267s
  MPS time: 0.2606s
  Speedup: 2.79x

Average MPS speedup: 1.07x
⚠️  Limited MPS acceleration - check PyTorch MPS installation


In [5]:
print("=== CHECKING PROJECT DEPENDENCIES ===")

required_packages = [
    "torch", "gpytorch", "numpy", "scipy", "matplotlib", 
    "networkx", "scikit-learn", "psutil"
]

missing_packages = []
for package in required_packages:
    try:
        __import__(package)
        print(f"✅ {package}")
    except ImportError:
        print(f"❌ {package} - MISSING")
        missing_packages.append(package)

if missing_packages:
    print(f"\n⚠️  Missing packages: {', '.join(missing_packages)}")
    print("Install with: pip install " + " ".join(missing_packages))
else:
    print("\n✅ All required packages are installed!")

=== CHECKING PROJECT DEPENDENCIES ===
✅ torch
✅ gpytorch
✅ numpy
✅ scipy✅ scipy
✅ matplotlib

✅ matplotlib
✅ networkx
❌ scikit-learn - MISSING
✅ psutil

⚠️  Missing packages: scikit-learn
Install with: pip install scikit-learn
✅ networkx
❌ scikit-learn - MISSING
✅ psutil

⚠️  Missing packages: scikit-learn
Install with: pip install scikit-learn


In [6]:
print("=== TESTING CUSTOM GRAPH KERNEL ===")

# Test importing the custom kernel
try:
    project_path = Path("/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs")
    if not project_path.exists():
        print("❌ Project path not found!")
        print(f"Expected: {project_path}")
        print("Please check the project location.")
    else:
        print(f"✅ Project path found: {project_path}")
        
    # Add to path
    sys.path.append(str(project_path))
    sys.path.append(str(project_path / "efficient_graph_gp_sparse"))
    
    # Try importing the kernel
    from efficient_graph_gp_sparse.gptorch_kernels_sparse.general_kernel_fast_grf import GraphGeneralFastGRFKernel
    print("✅ Successfully imported GraphGeneralFastGRFKernel")
    
    # Test basic kernel creation
    import scipy.sparse as sp
    import networkx as nx
    
    # Create a small test graph
    G = nx.cycle_graph(10)
    A = nx.adjacency_matrix(G).tocsr()
    
    print("\nTesting kernel creation...")
    kernel = GraphGeneralFastGRFKernel(
        adjacency_matrix=A,
        walks_per_node=1000,
        p_halt=0.1,
        max_walk_length=3,
        random_walk_seed=42
    )
    print("✅ Kernel created successfully")
    
    # Test kernel evaluation
    x1 = torch.tensor([[0], [1], [2]], dtype=torch.float32, device=device)
    x2 = torch.tensor([[0], [1]], dtype=torch.float32, device=device)
    
    print("Testing kernel evaluation...")
    with torch.no_grad():
        K = kernel(x1, x2)
        print(f"✅ Kernel evaluation successful - shape: {K.shape}")
        print(f"Kernel values range: [{K.min().item():.4f}, {K.max().item():.4f}]")
        
    # Check if using MPS optimization
    if hasattr(kernel, 'step_matrices_dense') and device.type == "mps":
        print("✅ Using MPS-optimized dense implementation")
        print(f"Step matrices on device: {kernel.step_matrices_dense.device}")
    elif hasattr(kernel, 'step_matrices'):
        print("✅ Using sparse implementation")
        print(f"Number of step matrices: {len(kernel.step_matrices)}")
    
except ImportError as e:
    print(f"❌ Failed to import custom kernel: {e}")
    print("Check that the project structure is correct and all files are present.")
except Exception as e:
    print(f"❌ Error testing kernel: {e}")

=== TESTING CUSTOM GRAPH KERNEL ===
✅ Project path found: /Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs
✅ Successfully imported GraphGeneralFastGRFKernel

Testing kernel creation...
Kernel initialized with 3 step matrices
Using device-aware dense tensors for MPS compatibility
✅ Kernel created successfully
Testing kernel evaluation...
✅ Kernel evaluation successful - shape: torch.Size([3, 2])
❌ Error testing kernel: 'LazyEvaluatedKernelTensor' object has no attribute 'min'
✅ Successfully imported GraphGeneralFastGRFKernel

Testing kernel creation...
Kernel initialized with 3 step matrices
Using device-aware dense tensors for MPS compatibility
✅ Kernel created successfully
Testing kernel evaluation...
✅ Kernel evaluation successful - shape: torch.Size([3, 2])
❌ Error testing kernel: 'LazyEvaluatedKernelTensor' object has no attribute 'min'


In [7]:
print("=== MEMORY AND PERFORMANCE TEST ===")

# Check current memory usage
memory_info = psutil.virtual_memory()
print(f"Current memory usage: {memory_info.percent:.1f}%")
print(f"Available memory: {memory_info.available / (1024**3):.1f} GB")

# Test with larger graph if kernel import was successful
try:
    from efficient_graph_gp_sparse.gptorch_kernels_sparse.general_kernel_fast_grf import GraphGeneralFastGRFKernel
    import gpytorch
    
    # Create larger test graph
    n_nodes = 100
    G = nx.barabasi_albert_graph(n_nodes, 3, seed=42)
    A = nx.adjacency_matrix(G).tocsr()
    
    print(f"\nTesting with {n_nodes} node graph...")
    print(f"Graph edges: {G.number_of_edges()}")
    print(f"Adjacency sparsity: {A.nnz / (n_nodes * n_nodes):.4f}")
    
    # Create kernel and model
    kernel = GraphGeneralFastGRFKernel(
        adjacency_matrix=A,
        walks_per_node=5000,
        p_halt=0.1,
        max_walk_length=3,
        random_walk_seed=42
    )
    
    # Test GP model creation
    n_train = 50
    train_idx = np.random.choice(n_nodes, size=n_train, replace=False)
    X_train = torch.tensor(train_idx, dtype=torch.float32).unsqueeze(1).to(device)
    y_train = torch.randn(n_train, device=device)
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood().to(device)
    
    class TestGPModel(gpytorch.models.ExactGP):
        def __init__(self, train_x, train_y, likelihood, kernel):
            super().__init__(train_x, train_y, likelihood)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(kernel)
            
        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    
    model = TestGPModel(X_train, y_train, likelihood, kernel).to(device)
    
    # Test forward pass timing
    model.eval()
    with torch.no_grad():
        start_time = time.time()
        output = model(X_train)
        forward_time = time.time() - start_time
        print(f"✅ Forward pass completed in {forward_time:.4f}s")
        
        # Test prediction on new points
        X_test = torch.tensor([[25], [75]], dtype=torch.float32, device=device)
        pred = likelihood(model(X_test))
        print(f"✅ Prediction completed - mean: {pred.mean.cpu().numpy()}")
    
    # Check memory usage after operations
    memory_after = psutil.virtual_memory()
    memory_used = memory_after.percent - memory_info.percent
    print(f"Additional memory used: {memory_used:.1f}%")
    
    if memory_used < 5.0:
        print("✅ Excellent memory efficiency!")
    elif memory_used < 15.0:
        print("✅ Good memory usage")
    else:
        print("⚠️  High memory usage - consider smaller graphs or batch processing")
        
except Exception as e:
    print(f"❌ Performance test failed: {e}")

=== MEMORY AND PERFORMANCE TEST ===
Current memory usage: 86.0%
Available memory: 1.1 GB

Testing with 100 node graph...
Graph edges: 291
Adjacency sparsity: 0.0582
Kernel initialized with 3 step matrices
Using device-aware dense tensors for MPS compatibility
Kernel initialized with 3 step matrices
Using device-aware dense tensors for MPS compatibility


/Users/matthew/Documents/Efficient Gaussian Process on Graphs/Efficient_Gaussian_Process_On_Graphs/venv/lib/python3.11/site-packages/gpytorch/models/exact_gp.py:296: GPInputWarning: The input matches the stored training data. Did you forget to call model.train()?
  warnings.warn(


✅ Forward pass completed in 0.1903s
✅ Prediction completed - mean: [-0.39313266  0.10191169]
Additional memory used: -6.0%
✅ Excellent memory efficiency!
✅ Prediction completed - mean: [-0.39313266  0.10191169]
Additional memory used: -6.0%
✅ Excellent memory efficiency!


In [8]:
print("=== OPTIMIZATION RECOMMENDATIONS ===")

# Check PyTorch settings
print("Current PyTorch settings:")
print(f"  torch.get_num_threads(): {torch.get_num_threads()}")
try:
    print(f"  MKL-DNN enabled: {torch.backends.mkldnn.is_available()}")
except:
    print("  MKL-DNN: Not available")

# Recommendations based on results
recommendations = []

if device.type == "cpu":
    recommendations.append("• Install PyTorch with MPS support for M2 acceleration:")
    recommendations.append("  pip install torch torchvision torchaudio")

if torch.get_num_threads() != psutil.cpu_count(logical=False):
    recommendations.append(f"• Consider setting torch.set_num_threads({psutil.cpu_count(logical=False)}) for optimal CPU performance")

memory_info = psutil.virtual_memory()
if memory_info.available < 4 * (1024**3):  # Less than 4GB available
    recommendations.append("• Low available memory - consider closing other applications")

if device.type == "mps":
    recommendations.append("• Using MPS device - optimal for M2 MacBook! 🚀")
    recommendations.append("• For large graphs (>1000 nodes), consider batch processing")
    recommendations.append("• MPS works best with float32 tensors")

if not recommendations:
    recommendations.append("• Your setup looks optimal! 🎉")

print("\n" + "="*50)
print("RECOMMENDATIONS:")
for rec in recommendations:
    print(rec)
print("="*50)

=== OPTIMIZATION RECOMMENDATIONS ===
Current PyTorch settings:
  torch.get_num_threads(): 4
  MKL-DNN enabled: False

RECOMMENDATIONS:
• Consider setting torch.set_num_threads(8) for optimal CPU performance
• Low available memory - consider closing other applications
• Using MPS device - optimal for M2 MacBook! 🚀
• For large graphs (>1000 nodes), consider batch processing
• MPS works best with float32 tensors


## Summary

This compatibility check verifies:

✅ **System Requirements**: Python, platform, and hardware details  
✅ **PyTorch Installation**: Version, CUDA/MPS support, and device selection  
✅ **GPU Acceleration**: Tensor operations benchmarking and speedup measurement  
✅ **Dependencies**: All required packages for the project  
✅ **Custom Kernels**: Import and basic functionality testing  
✅ **Performance**: Memory usage and execution timing  
✅ **Optimization**: Settings and recommendations for best performance  

If all checks pass with ✅, your M2 MacBook is ready to run the Efficient Gaussian Process on Graphs project with optimal performance!

### Next Steps:
1. If any issues were found, follow the recommendations above
2. Run the main experiment notebooks in `/experiments_sparse/`
3. For large-scale experiments, monitor memory usage and consider batch processing